## Problem 1 [25 points]

Create an abstract base class `Expression` that represents a mathematical expression. This class should require methods for evaluating the expression (given a dictionary of variable values), for computing its symbolic derivative with respect to a variable. Also, it should require pretty printing and formatting for expressions. Then, implement concrete classes for constants, variables and binary operations (addition, subtraction, multiplication and division).



In [1]:
from abc import ABC, abstractmethod

class Expression(ABC):
    @abstractmethod
    def evaluate(self, variables: dict):
        pass

    @abstractmethod
    def derivative(self, x):
        pass

    @abstractmethod
    def __repr__(self):
        pass

class Constant(Expression):
    def __init__(self, value):
        self._value = value

    def evaluate(self, variables):
        return self._value

    def derivative(self, x):
        return Constant(0)

    def __repr__(self):
        return str(self._value)

class Variable(Expression):
    def __init__(self, name):
        self._name = name

    def evaluate(self, variables):
        return variables[self._name]

    def derivative(self, x):
      if self._name == x:
          return Constant(1)
      return Constant(0)

    def __repr__(self):
        return self._name

class Add(Expression):
  def __init__(self, l, r):
    self._l = l
    self._r = r

  def evaluate(self, variables):
    return self._l.evaluate(variables) + self._r.evaluate(variables)

  def derivative(self, x):
    return Add(self._l.derivative(x), self._r.derivative(x))

  def __repr__(self):
    return f"({str(self._l)} + {str(self._r)})"

class Subtract(Expression):
  def __init__(self, l, r):
    self._l = l
    self._r = r

  def evaluate(self, variables):
    return self._l.evaluate(variables) - self._r.evaluate(variables)

  def derivative(self, x):
    return Subtract(self._l.derivative(x), self._r.derivative(x))

  def __repr__(self):
    return f"({str(self._l)} - {str(self._r)})"


class Multiply(Expression):
  def __init__(self, l, r):
    self._l = l
    self._r = r

  def evaluate(self, variables):
    return self._l.evaluate(variables) * self._r.evaluate(variables)

  def derivative(self, x):
    return Add(Multiply(self._l.derivative(x), self._r), Multiply(self._l, self._r.derivative(x)))

  def __repr__(self):
    return f"{str(self._l)} * {str(self._r)}"


class Divide(Expression):
  def __init__(self, l, r):
    self._l = l
    self._r = r

  def evaluate(self, variables):
    return self._l.evaluate(variables) / self._r.evaluate(variables)

  def derivative(self, x):
    return Divide(Subtract(Multiply(self._l.derivative(x), self._r),
                  Multiply(self._l, self._r.derivative(x))), Multiply(self._r, self._r))

  def __repr__(self):
    return f"{str(self._l)} / {str(self._r)}"

In [2]:
expr = Divide(
    Multiply(
        Add(Multiply(Constant(2), Variable("x")), Constant(3)),
        Subtract(Variable("x"), Constant(5))
    ),
    Subtract(Constant(1), Variable("x"))
)

print(expr) # (2 * x + 3) * (x - 5) / (1 - x)

assert expr.evaluate({"x": 3}) == 9.0

derivative_expr = expr.derivative("x")

assert derivative_expr.evaluate({"x": 3}) == -7.0

(2 * x + 3) * (x - 5) / (1 - x)


## Problem 2 [25 points]

A Binary Search Tree (BST) is a data structure that consists of nodes, where each node contains a value and references to two child nodes: left and right. The key property of a BST is that for every node:

- The value of the left child node is less than the value of the parent node.
- The value of the right child node is greater than the value of the parent node.

Implement a generic BST that supports any comparable type (integers, floats, strings, custom objects). It should provide standard `insert` and `search` operations as defined below:

- **Insertion:** Insert a new node while maintaining the BST property.
- **Search:** Find a node in the tree based on its value.

In [3]:
from typing import TypeVar, Optional, Generic

T = TypeVar('T')

class BSTNode(Generic[T]):
    def __init__(self, x: T):
        self._x: T = x
        self._left: Optional[BSTNode[T]] = None
        self._right: Optional[BSTNode[T]] = None

class BST(Generic[T]):
    def __init__(self):
        self._start: Optional[BSTNode[T]] = None

    def insert(self, item: T) -> None:
        node = self._start
        if node is None:
            self._start = BSTNode(item)
            return

        while True:
            if item < node._x:
                if node._left is None:
                    node._left = BSTNode(item)
                    return
                node = node._left
            elif item > node._x:
                if node._right is None:
                    node._right = BSTNode(item)
                    return
                node = node._right
            else:
                return

    def search(self, target: T) -> bool:
        node = self._start
        while node is not None:
            if target == node._x:
                return True
            elif target < node._x:
                node = node._left
            else:
                node = node._right
        return False

In [4]:
"""
        10
       /  \
     5     15
    / \    /  \
   3   7  12   20
"""
tree = BST[int]()
tree.insert(10)
tree.insert(5)
tree.insert(15)
tree.insert(3)
tree.insert(7)
tree.insert(12)
tree.insert(20)

assert tree.search(7) == True
assert tree.search(15) == True
assert tree.search(42) == False

## Problem 3 [25 points]

a. Implement a descriptor that tracks the history of all values assigned to an attribute.

In [5]:
class HistoryTracking:
    def __set_name__(self, owner, name):
        print(f"__set_name__ called: owner={owner}, name={name}")
        self._name = name
        self._history_name = f"_{name}_history"

    def __get__(self, instance, owner):
        return instance.__dict__.get(self._name, None)

    def __set__(self, instance, value):
        if self._name in instance.__dict__:
            history = instance.__dict__.setdefault(self._history_name, [])
            history.append(instance.__dict__[self._name])
        else:
            instance.__dict__[self._history_name] = []
        instance.__dict__[self._name] = value

In [6]:
class StockPrice:
    price = HistoryTracking()

    def __init__(self, name, price):
        self.name = name
        self.price = price

    def get_price_history(self):
        return self.__dict__.get('_price_history', [])

__set_name__ called: owner=<class '__main__.StockPrice'>, name=price


In [7]:
stock = StockPrice("AAPL", 200)

assert stock.price == 200

stock.price = 210

assert stock.price == 210

stock.price = 225

assert stock.price == 225
assert stock.get_price_history() == [200, 210]

stock.price = 215

assert stock.price == 215
assert stock.get_price_history() == [200, 210, 225]

b. Implement a descriptor that logs the access to an attribute, along with the time and value, into a log file.


In [8]:
import time

class FileLoggingDescriptor:
    def __init__(self, log_file):
        self._log_file = log_file

    def __set_name__(self, owner, name):
        print(f"__set_name__ called: owner={owner}, name={name}")
        self._name = name

    def __get__(self, instance, owner):
        value = instance.__dict__.get(self._name, "no value")
        with open(self._log_file, "a") as f:
            f.write(f"{time.ctime()}: Accessed value '{value}'\n")
        return value

    def __set__(self, instance, value):
        old_value = instance.__dict__.get(self._name, "no value")
        instance.__dict__[self._name] = value
        with open(self._log_file, "a") as f:
            f.write(f"{time.ctime()}: Modified value from '{old_value}' to '{value}'\n")

In [9]:
class Person:
    name = FileLoggingDescriptor(log_file="access_log.txt")

    def __init__(self, name):
        self.name = name

__set_name__ called: owner=<class '__main__.Person'>, name=name


In [10]:
obj = Person("John Doe")

print(obj.name) # "John Doe"

obj.name = "Jane Dane"

print(obj.name) # "Jane Dane"

with open("access_log.txt", "r") as f:
    print(f.read())

John Doe
Jane Dane
Tue Apr 15 15:53:07 2025: Modified value from 'no value' to 'John Doe'
Tue Apr 15 15:53:07 2025: Accessed value 'John Doe'
Tue Apr 15 15:53:07 2025: Modified value from 'John Doe' to 'Jane Dane'
Tue Apr 15 15:53:07 2025: Accessed value 'Jane Dane'



## Problem 4 [25 points]

Write a function `build_logging_class` that dynamically generates a new class which wraps all public methods of a given base class with logging behavior.

The logging should:

- Print a log message before the method is called, showing its name and arguments.
- Print a log message after the method is called, showing the result.
- Exclude dunder methods (`__init__`, `__str__`, etc.).
- Allow injection of a custom logging function (defaults to print).

#### (Optional) Bonus requirements

- Support a logging level prefix like [ERROR], [INFO], [DEBUG].
- Allow logging to be written to a file.
- Support toggling logging on/off.



In [11]:
from typing import Callable
from functools import wraps

def build_logging_class(class_name: str, base_class: type, logger: Callable = print) -> type:
    wrapped_methods = {}

    for method_name in dir(base_class):
        if method_name.startswith("_"):
            continue

        method = getattr(base_class, method_name)

        if callable(method):
            def wrap_method(func, name=method_name):
                @wraps(func)
                def logged_method(self, *args, **kwargs):
                    logger(f"[LOG] Calling: `{name}` with args={args} kwargs={kwargs}")
                    result = func(self, *args, **kwargs)
                    logger(f"[LOG] Result of `{name}`: {result}")
                    return result
                return logged_method

            wrapped_methods[method_name] = wrap_method(method)

    return type(class_name, (base_class,), wrapped_methods)

In [12]:
class Calculator:
    def add(self, x, y): return x + y
    def mul(self, x, y): return x * y

LoggingCalculator = build_logging_class("LoggingCalculator", Calculator)

calc = LoggingCalculator()
calc.add(2, 3)
calc.mul(4, 5)

"""
Expected output

[LOG] Calling: `add` with args=(2, 3) kwargs={}
[LOG] Result of `add`: 5
[LOG] Calling: `mul` with args=(4, 5) kwargs={}
[LOG] Result of `mul`: 20
"""

[LOG] Calling: `add` with args=(2, 3) kwargs={}
[LOG] Result of `add`: 5
[LOG] Calling: `mul` with args=(4, 5) kwargs={}
[LOG] Result of `mul`: 20


'\nExpected output\n\n[LOG] Calling: `add` with args=(2, 3) kwargs={}\n[LOG] Result of `add`: 5\n[LOG] Calling: `mul` with args=(4, 5) kwargs={}\n[LOG] Result of `mul`: 20\n'